# Casualties and Migration in the Syrian Civil War

## Introduction

In 2011, five weeks into the civil demonstrations against the Syrian government, secret police forces detained and tortured fifteen students who had spray painted an anti-government statement on the walls of their school. They would be released weeks later in an effort to quell the rising civil unrest in the province. In the wake of the hundreds of other demonstrators who were killed or disappeared, this action was too little and too late to stop the tide of the civil war. Demonstrations turned to protest turned to armed conflict and the rest is history.

The war would go on to spawn both the largest refugee crisis and one of the deadliest conflicts in modern history. As of 2019, there are over 6 million Syrian refugees and another 6 million internally displaced people in a country with a pre-war population of around 24 million (UNHCR, 2018). The regime's efforts to prevent accurate information from leaving the country has made it nearly impossible to estimate the number of casualties that have occured in that time. Current estimates range from 300,000 to 600,000 killed depending on the source.

The link between the flow of violence within the country and the flow of asylum seekers out of the country should be apparent to anyone who is aware of the war. Yet a growing sentiment among residents in host countries is that a large portion of asylum seekers from Syria are actually economic migrants, who are using the conflict as a means of gaining entry into the European Union and access to generous social programs.

We believe that violence is the most important predictor of migration of Syrian refugees; however, while this argument may be generally accepted, there is great difficulty in proving this relationship for certain. We hope to answer this question using reported casualty data to see whether there is a correlation between violence in a given province and a subsequent increase in the amount of asylum seekers across all host countries.

## Project

Our project can be organized into three distinct portions:

1. Data Scraping
2. Data Wrangling
3. Data Visualization

Our goal is to create a dataset for casualty information a refugee data, clean and structure the dataset for easier queying, and visualize the data to provide more insights into the questions we pose above. 

## Data Scraping

There are multiple sources that could be used for casualty information (list here). We will leave the three datasets for now, and focus on the VDC and CSR datasets because they provide their data is table elements that make it easy for us to scrape and organize our dataframes for analysis.

We will now go through the process of scraping and creating the inital forms of these datasets.

### Casualty Data

#### VDC

#### CSR

### Refugee Data

#### Monthly Inflows

#### Yearly Refugee Status

## Data Wrangling

### Casualty Data

#### VDC

#### CSR

### Refugee Data

#### Monthly Inflows

#### Yearly Refugee Status

## Data Visualization

### Casualty Data

#### VDC

#### CSR

### Refugee Data

#### Monthly Inflows

#### Yearly Refugee Status

## Future Work

## Conclusion

## Appendix

## References